In [41]:
import numpy as np
import pandas as pd
import os

In [42]:
PATH = ''

train = pd.read_csv('./training_v2.csv')
test = pd.read_csv('./unlabeled.csv')

In [43]:
train["apache_4a_hospital_death_prob"]=train["apache_4a_hospital_death_prob"].replace({-1:np.nan})
test["apache_4a_hospital_death_prob"]=test["apache_4a_hospital_death_prob"].replace({-1:np.nan})

train["apache_4a_icu_death_prob"]=train["apache_4a_icu_death_prob"].replace({-1:np.nan})
test["apache_4a_icu_death_prob"]=test["apache_4a_icu_death_prob"].replace({-1:np.nan})


In [44]:
train.loc[train['pre_icu_los_days'] <0, 'pre_icu_los_days'] = np.nan
test.loc[test['pre_icu_los_days'] <0, 'pre_icu_los_days'] = np.nan

In [45]:
#train.drop(['encounter_id', 'patient_id', 'readmission_status'], axis=1, inplace=True)
#test.drop(['encounter_id', 'patient_id', 'readmission_status'], axis=1, inplace=True)

New variables

In [46]:
train['apache_3j_diagnosis_root'] = [str(element).split('.')[0] for element in train['apache_3j_diagnosis'] ]
test['apache_3j_diagnosis_root'] = [str(element).split('.')[0] for element in test['apache_3j_diagnosis'] ]

In [47]:
#Creation of sepsis stuff
#train
train['temp_flag1'] = [temp>38.0 or temp<35.0 for temp in train['temp_apache']]  
train['temp_flag2'] = [temp>38.0 or temp<35.0 for temp in train['d1_temp_max']] 
train['temp_flag3'] = [temp>38.0 or temp<35.0 for temp in train['h1_temp_max']] 
train['temp_flag4'] = [temp>38.0 or temp<35.0 for temp in train['d1_temp_min']] 
train['temp_flag5'] = [temp>38.0 or temp<35.0 for temp in train['h1_temp_min']]
train['temp_flag'] = (train.temp_flag1 | train.temp_flag2 | train.temp_flag3 | train.temp_flag4 | train.temp_flag5)  

train['heart_rate_flag'] = (train.heart_rate_apache > 90.0) | (train.d1_heartrate_max > 90.0) | (train.d1_heartrate_min >90.0)

train['resp_flag1'] = (train.resprate_apache >20.0) | (train.d1_resprate_max > 20.0) | (train.d1_resprate_min >20.0) | (train.h1_resprate_max >20.0) | (train.h1_resprate_min >20.0)
train['resp_flag2'] = (train.paco2_apache < 32.0) | (train.d1_arterial_pco2_max < 32.0) | (train.d1_arterial_pco2_min < 32.0) | (train.h1_arterial_pco2_max < 32.0) | (train.h1_arterial_pco2_min < 32.0)
 

train['resp_flag'] = (train.resp_flag1 | train.resp_flag2)

train['wbc_apache_flag1'] = [wb>12.0 or wb<4.0 for wb in train['wbc_apache']]
train['wbc_apache_flag2'] = [wb>12.0 or wb<4.0 for wb in train['d1_wbc_max']]
train['wbc_apache_flag3'] = [wb>12.0 or wb<4.0 for wb in train['d1_wbc_min']]
train['wbc_apache_flag4'] = [wb>12.0 or wb<4.0 for wb in train['h1_wbc_max']]
train['wbc_apache_flag5'] = [wb>12.0 or wb<4.0 for wb in train['h1_wbc_min']]


train['wbc_apache_flag'] = (train.wbc_apache_flag1 | train.wbc_apache_flag2 | train.wbc_apache_flag3 | train.wbc_apache_flag4 | train.wbc_apache_flag5) 




In [48]:
#Correction NA scores
train.loc[(train.temp_apache.isna() & train.d1_temp_max.isna() & train.h1_temp_max.isna() & train.d1_temp_min & train.h1_temp_min.isna()), 'temp_flag'] =np.nan
train.loc[(train.heart_rate_apache.isna() & train.d1_heartrate_max.isna() & train.d1_heartrate_min.isna()), 'heart_rate_flag'] =np.nan
train.loc[train.resprate_apache.isna() & train.d1_resprate_max.isna() & train.d1_resprate_min.isna() & train.h1_resprate_max.isna() & train.h1_resprate_min.isna() & train.paco2_apache.isna() & train.d1_arterial_pco2_max.isna() & train.d1_arterial_pco2_min.isna() & train.h1_arterial_pco2_max.isna() & train.h1_arterial_pco2_min.isna(), 'resp_flag'] =np.nan
train.loc[train.wbc_apache.isna() & train.d1_wbc_max.isna() & train.d1_wbc_min.isna() & train.h1_wbc_max.isna() & train.h1_wbc_min.isna(), 'wbc_apache_flag'] =np.nan

In [49]:
train['SIRS'] = train['temp_flag'] *1 + train['heart_rate_flag'] * 1 + train['resp_flag'] * 1 + train['wbc_apache_flag'] * 1

In [50]:
test['temp_flag1'] = [temp>38.0 or temp<35.0 for temp in test['temp_apache']]  
test['temp_flag2'] = [temp>38.0 or temp<35.0 for temp in test['d1_temp_max']] 
test['temp_flag3'] = [temp>38.0 or temp<35.0 for temp in test['h1_temp_max']] 
test['temp_flag4'] = [temp>38.0 or temp<35.0 for temp in test['d1_temp_min']] 
test['temp_flag5'] = [temp>38.0 or temp<35.0 for temp in test['h1_temp_min']]
test['temp_flag'] = (test.temp_flag1 | test.temp_flag2 | test.temp_flag3 | test.temp_flag4 | test.temp_flag5)  

test['heart_rate_flag'] = (test.heart_rate_apache > 90.0) | (test.d1_heartrate_max > 90.0) | (test.d1_heartrate_min >90.0)

test['resp_flag1'] = (test.resprate_apache >20.0) | (test.d1_resprate_max > 20.0) | (test.d1_resprate_min >20.0) | (test.h1_resprate_max >20.0) | (test.h1_resprate_min >20.0)
test['resp_flag2'] = (test.paco2_apache < 32.0) | (test.d1_arterial_pco2_max < 32.0) | (test.d1_arterial_pco2_min < 32.0) | (test.h1_arterial_pco2_max < 32.0) | (test.h1_arterial_pco2_min < 32.0)
 

test['resp_flag'] = (test.resp_flag1 | test.resp_flag2)

test['wbc_apache_flag1'] = [wb>12.0 or wb<4.0 for wb in test['wbc_apache']]
test['wbc_apache_flag2'] = [wb>12.0 or wb<4.0 for wb in test['d1_wbc_max']]
test['wbc_apache_flag3'] = [wb>12.0 or wb<4.0 for wb in test['d1_wbc_min']]
test['wbc_apache_flag4'] = [wb>12.0 or wb<4.0 for wb in test['h1_wbc_max']]
test['wbc_apache_flag5'] = [wb>12.0 or wb<4.0 for wb in test['h1_wbc_min']]


test['wbc_apache_flag'] = (test.wbc_apache_flag1 | test.wbc_apache_flag2 | test.wbc_apache_flag3 | test.wbc_apache_flag4 | test.wbc_apache_flag5) 


In [51]:
#Correction NA scores
test.loc[(test.temp_apache.isna() & test.d1_temp_max.isna() & test.h1_temp_max.isna() & test.d1_temp_min & test.h1_temp_min.isna()), 'temp_flag'] = np.nan
test.loc[(test.heart_rate_apache.isna() & test.d1_heartrate_max.isna() & test.d1_heartrate_min.isna()), 'heart_rate_flag'] = np.nan
test.loc[test.resprate_apache.isna() & test.d1_resprate_max.isna() & test.d1_resprate_min.isna() & test.h1_resprate_max.isna() & test.h1_resprate_min.isna() & test.paco2_apache.isna() & test.d1_arterial_pco2_max.isna() & test.d1_arterial_pco2_min.isna() & test.h1_arterial_pco2_max.isna() & test.h1_arterial_pco2_min.isna(), 'resp_flag'] =np.nan
test.loc[test.wbc_apache.isna() & test.d1_wbc_max.isna() & test.d1_wbc_min.isna() & test.h1_wbc_max.isna() & test.h1_wbc_min.isna(), 'wbc_apache_flag'] =np.nan

In [52]:
test.heart_rate_flag.isna().sum()

106

In [53]:
test['SIRS'] = test['temp_flag'] *1 + test['heart_rate_flag'] * 1 + test['resp_flag'] * 1 + test['wbc_apache_flag'] * 1

In [54]:
#drop auxiliary flags
train.drop(['temp_flag1', 'temp_flag2', 'temp_flag3', 'temp_flag4', 'temp_flag5', 'resp_flag1', 'resp_flag2', 'wbc_apache_flag1', 'wbc_apache_flag2', 'wbc_apache_flag3', 'wbc_apache_flag4', 'wbc_apache_flag5'], axis=1, inplace=True)
test.drop(['temp_flag1', 'temp_flag2', 'temp_flag3', 'temp_flag4', 'temp_flag5', 'resp_flag1', 'resp_flag2', 'wbc_apache_flag1', 'wbc_apache_flag2', 'wbc_apache_flag3', 'wbc_apache_flag4', 'wbc_apache_flag5'], axis=1, inplace=True)

In [55]:
train['Glasglow_comma_score'] = train['gcs_eyes_apache'] + train['gcs_motor_apache'] + train['gcs_verbal_apache']
test['Glasglow_comma_score'] = test['gcs_eyes_apache'] + test['gcs_motor_apache'] + test['gcs_verbal_apache']

In [56]:
#https://bvns.net/bvns-neurotransmitter-2-0-technically-speaking-september-2016/
def Glasglow_comma_score_t(x): 
    if pd.isna(x):
        return np.nan
    elif x<=3.:
        return 'Unresponsive'
    elif x > 3. and x <= 8.: 
        return 'Comatose' 
    elif x > 8.  and x <= 14.: 
        return 'Guarded'
    else:
        return 'Best_score'

train['Glasglow_comma_score_t'] = train['Glasglow_comma_score'].map(Glasglow_comma_score_t)
test['Glasglow_comma_score_t'] = test['Glasglow_comma_score'].map(Glasglow_comma_score_t)

Recalculate BMI thanks to this comment

In [57]:
train['height'] = train['height']/100
test['height'] = test['height']/100

In [58]:
train['bmi'] = train['weight']/(train['height']**2)
test['bmi'] = test['weight']/(test['height']**2)

Thanks to this comment https://www.kaggle.com/c/widsdatathon2020/discussion/127987

In [59]:
def weighted_classt(x): 
    if pd.isna(x):
        return np.nan
    elif x < 15: 
        return 'very severely underweight' 
    elif x >= 15 and x < 16: 
        return 'severely weight' 
    elif x >=16 and x < 18.5: 
        return 'underweight' 
    elif x >= 18.5 and x < 25: 
        return 'healthy weight' 
    elif x >= 25 and x < 30: 
        return 'overweight'
    elif x >= 30 and x < 35: 
        return 'class 1' 
    elif x >= 35 and x < 40: 
        return 'class 2' 
    else: 
        return 'class 3' 

train['weightclass'] = train['bmi'].map(weighted_classt)
test['weightclass'] = test['bmi'].map(weighted_classt)

In [60]:
#check counts

In [61]:
#Delete features

pd.crosstab(train.apache_3j_bodysystem, train.apache_2_bodysystem)

apache_2_bodysystem,Cardiovascular,Gastrointestinal,Haematologic,Metabolic,Neurologic,Renal/Genitourinary,Respiratory,Trauma,Undefined Diagnoses,Undefined diagnoses
apache_3j_bodysystem,,,,,,,,,,
Cardiovascular,27051,0,0,0,0,0,0,0,0,2948
Gastrointestinal,0,9026,0,0,0,0,0,0,0,0
Genitourinary,25,0,0,0,0,2147,0,0,0,0
Gynecological,0,0,0,0,0,313,0,0,0,0
Hematological,0,0,638,0,0,0,0,0,0,0
Metabolic,0,0,0,7650,0,0,0,0,0,0
Musculoskeletal/Skin,0,0,0,0,0,0,0,0,346,820
Neurological,0,0,0,0,11896,0,0,0,0,0
Respiratory,0,0,0,0,0,0,11609,0,0,0


apache_3j_bodysystem is more detailed and almost the same variable as apache_2_bodysystem so we remove the latter.

In [62]:
train.drop(['apache_2_bodysystem'], axis=1, inplace=True)
test.drop(['apache_2_bodysystem'], axis=1, inplace=True)

In [63]:

#previous icu length of stay

train['icu_los_flag'] = 0
train.loc[train.pre_icu_los_days >15, 'icu_los_flag'] = 1
train.loc[train.pre_icu_los_days.isna(), 'icu_los_flag'] = np.nan

In [64]:
test['icu_los_flag'] = 0
test.loc[test.pre_icu_los_days >15, 'icu_los_flag'] = 1
test.loc[test.pre_icu_los_days.isna(), 'icu_los_flag'] = np.nan

In [65]:
#From apache 3 definition this interaction is significant
train['paco2_apache_ph_apache_x'] = train['paco2_apache'] * train['ph_apache']
test['paco2_apache_ph_apache_x'] = test['paco2_apache'] * test['ph_apache']

In [66]:
train['paco2_ph_d1max_x'] = train.d1_arterial_po2_max * train.d1_arterial_ph_max
train['paco2_ph_d1min_x'] = train.d1_arterial_po2_min * train.d1_arterial_ph_min
train['paco2_ph_h1max_x'] = train.h1_arterial_po2_max * train.h1_arterial_ph_max
train['paco2_ph_h1min_x'] = train.h1_arterial_po2_min * train.h1_arterial_ph_min



test['paco2_ph_d1max_x'] = test.d1_arterial_po2_max * test.d1_arterial_ph_max
test['paco2_ph_d1min_x'] = test.d1_arterial_po2_min * test.d1_arterial_ph_min
test['paco2_ph_h1max_x'] = test.h1_arterial_po2_max * test.h1_arterial_ph_max
test['paco2_ph_h1min_x'] = test.h1_arterial_po2_min * test.h1_arterial_ph_min


In [67]:
train['heart_rate_apache'].describe()

count    90835.000000
mean        99.707932
std         30.870502
min         30.000000
25%         86.000000
50%        104.000000
75%        120.000000
max        178.000000
Name: heart_rate_apache, dtype: float64

In [68]:
#Apache 3 covariates

In [69]:
#Pulse apache
def pulse(x): 
    if pd.isna(x):
        return np.nan
    elif x <= 39.:
        return 8
    elif x > 39. and x <= 49.: 
        return 5 
    elif x > 49.  and x <= 99.: 
        return 0
    elif x > 99. and x <= 109.:
        return 1
    elif x >109. and x<= 119.:
        return 5
    elif x >119 and x<= 139.:
        return 7
    elif x > 139. and x<=154:
        return 13
    elif x > 154:
        return 17
    else:
        return 'Best_score'

train['pulse_apache'] = train['heart_rate_apache'].map(pulse)
test['pulse_apache'] = test['heart_rate_apache'].map(pulse)



# Mean blood pressure

def map_apache(x): 
    if pd.isna(x):
        return np.nan
    elif x <= 39.:
        return 23
    elif x > 39. and x <= 59.: 
        return 15 
    elif x > 59.  and x <= 69.: 
        return 7
    elif x > 69. and x <= 79.:
        return 6
    elif x >79. and x<= 99.:
        return 0
    elif x >99. and x<= 119.:
        return 4
    elif x > 119. and x<=129:
        return 7
    elif x > 129. and x<=139:
        return 9
    elif x > 139:
        return 10
    else:
        return 'Best_score'


train['meanBP_apache'] = train['map_apache'].map(map_apache)
test['meanBP_apache'] = test['map_apache'].map(map_apache)

#Temperature

def temperature(x): 
    if pd.isna(x):
        return np.nan
    elif x <= 32.9:
        return 20
    elif x > 32.9 and x <= 33.4: 
        return 16 
    elif x > 33.4  and x <= 33.9: 
        return 13
    elif x > 33.9 and x <= 34.9:
        return 8
    elif x >34.9 and x<= 35.9:
        return 2
    elif x >35.9 and x<= 36.9:
        return 0
    elif x >36.9 and x< 40:
        return 2 #Not defined in the graph I have ?? this is slight fever    
    elif x >=40:
        return 10
    else:
        return 'Best_score'

train['temperature_apache'] = train['temp_apache'].map(temperature)
test['temperature_apache'] = test['temp_apache'].map(temperature)


#respiratory_rate


def resprate_apache(x): 
    if pd.isna(x):
        return np.nan
    elif x <= 5:
        return 17
    elif x > 5 and x <= 11: 
        return 8 
    elif x > 11  and x <= 13: 
        return 7
    elif x > 13 and x <= 24:
        return 0
    elif x > 24 and x<= 34:
        return 6
    elif x > 34 and x<= 39:
        return 9
    elif x > 39 and x<= 49:
        return 11   
    elif x > 49:
        return 18
    else:
        return 'Best_score'



train['rr_apache'] = train['resprate_apache'].map(resprate_apache)
test['rr_apache'] = test['resprate_apache'].map(resprate_apache)

#PaO2

def pao2_apache(x): 
    if pd.isna(x):
        return np.nan
    elif x <= 5:
        return 17
    elif x > 5 and x <= 11: 
        return 8 
    elif x > 11  and x <= 13: 
        return 7
    elif x > 13 and x <= 24:
        return 0
    elif x > 24 and x<= 34:
        return 6
    elif x > 34 and x<= 39:
        return 9
    elif x > 39 and x<= 49:
        return 11   
    elif x > 49:
        return 18
    else:
        return 'Best_score'

#


#AaDO2 ?????/
#Hematocrit 

def hematocrit_apache(x): 
    if pd.isna(x):
        return np.nan
    elif x <= 5:
        return 17
    elif x > 5 and x <= 11: 
        return 8 
    elif x > 11  and x <= 13: 
        return 7
    elif x > 13 and x <= 24:
        return 0
    elif x > 24 and x<= 34:
        return 6
    elif x > 34 and x<= 39:
        return 9
    elif x > 39 and x<= 49:
        return 11   
    elif x > 49:
        return 18
    else:
        return 'Best_score'

#WBC_count 

def wbc_apache(x): 
    if pd.isna(x):
        return np.nan
    elif x <= 5:
        return 17
    elif x > 5 and x <= 11: 
        return 8 
    elif x > 11  and x <= 13: 
        return 7
    elif x > 13 and x <= 24:
        return 0
    elif x > 24 and x<= 34:
        return 6
    elif x > 34 and x<= 39:
        return 9
    elif x > 39 and x<= 49:
        return 11   
    elif x > 49:
        return 18
    else:
        return 'Best_score'

#Serum creatinine creatinine_apache

#Pulse apache
def pulse(x): 
    if pd.isna(x):
        return np.nan
    elif x <= 39.:
        return 8
    elif x > 39. and x <= 49.: 
        return 5 
    elif x > 49.  and x <= 99.: 
        return 0
    elif x > 99. and x <= 109.:
        return 1
    elif x >109. and x<= 119.:
        return 5
    elif x >119 and x<= 139.:
        return 7
    elif x > 139. and x<=154:
        return 13
    elif x > 154:
        return 17
    else:
        return 'Best_score'

train['pulse_apache'] = train['heart_rate_apache'].map(pulse)
test['pulse_apache'] = test['heart_rate_apache'].map(pulse)



# Mean blood pressure

def map_apache(x): 
    if pd.isna(x):
        return np.nan
    elif x <= 39.:
        return 23
    elif x > 39. and x <= 59.: 
        return 15 
    elif x > 59.  and x <= 69.: 
        return 7
    elif x > 69. and x <= 79.:
        return 6
    elif x >79. and x<= 99.:
        return 0
    elif x >99. and x<= 119.:
        return 4
    elif x > 119. and x<=129:
        return 7
    elif x > 129. and x<=139:
        return 9
    elif x > 139:
        return 10
    else:
        return 'Best_score'


train['meanBP_apache'] = train['map_apache'].map(map_apache)
test['meanBP_apache'] = test['map_apache'].map(map_apache)

#Temperature

def temperature(x): 
    if pd.isna(x):
        return np.nan
    elif x <= 32.9:
        return 20
    elif x > 32.9 and x <= 33.4: 
        return 16 
    elif x > 33.4  and x <= 33.9: 
        return 13
    elif x > 33.9 and x <= 34.9:
        return 8
    elif x >34.9 and x<= 35.9:
        return 2
    elif x >35.9 and x<= 36.9:
        return 0
    elif x >36.9 and x< 40:
        return 2 #Not defined in the graph I have ?? this is slight fever    
    elif x >=40:
        return 10
    else:
        return 'Best_score'

train['temperature_apache'] = train['temp_apache'].map(temperature)
test['temperature_apache'] = test['temp_apache'].map(temperature)


#respiratory_rate


def resprate_apache(x): 
    if pd.isna(x):
        return np.nan
    elif x <= 5:
        return 17
    elif x > 5 and x <= 11: 
        return 8 
    elif x > 11  and x <= 13: 
        return 7
    elif x > 13 and x <= 24:
        return 0
    elif x > 24 and x<= 34:
        return 6
    elif x > 34 and x<= 39:
        return 9
    elif x > 39 and x<= 49:
        return 11   
    elif x > 49:
        return 18
    else:
        return 'Best_score'

train['rr_apache'] = train['resprate_apache'].map(resprate_apache)
test['rr_apache'] = test['resprate_apache'].map(resprate_apache)

#PaO2

def pao2_apache(x): 
    if pd.isna(x):
        return np.nan
    elif x <= 49:
        return 15
    elif x > 49 and x <= 69: 
        return 5 
    elif x > 69  and x <= 79: 
        return 2
    elif x > 79:
        return 0
    else:
        return 'Best_score'

train['pao2_apache_s'] = train['pao2_apache'].map(pao2_apache)
test['pao2_apache_s'] = test['pao2_apache'].map(pao2_apache)


In [70]:
#AaDO2 ?????/
#Hematocrit 

def hematocrit_apache(x): 
    if pd.isna(x):
        return np.nan
    elif x <= 40.9:
        return 3
    elif x > 40.9 and x <= 49: 
        return 0 
    elif x > 49: 
        return 3
    else:
        return 'Best_score'

    
train['hematocrit_apache_s'] = train['hematocrit_apache'].map(hematocrit_apache)
test['hematocrit_apache_s'] = test['hematocrit_apache'].map(hematocrit_apache)    
    
    
    
#WBC_count 

def wbc_apache(x): 
    if pd.isna(x):
        return np.nan
    elif x <= 1.:
        return 19
    elif x > 1. and x <= 2.9: 
        return 5 
    elif x > 2.9  and x <= 19.9: 
        return 0
    elif x > 19.9 and x <= 24.9:
        return 1
    elif x > 24.9:
        return 5
    else:
        return 'Best_score'

train['wbc_apache_s'] = train['wbc_apache'].map(wbc_apache)
test['wbc_apache_s'] = test['wbc_apache'].map(wbc_apache)    
    
    
#Serum creatinine 

def creatinine_apache(x): 
    if pd.isna(x):
        return np.nan
    elif x <= 0.4:
        return 3
    elif x > 0.4 and x <= 1.4: 
        return 0 
    elif x > 1.4  and x <= 1.94: 
        return 4
    elif x > 1.94:
        return 7
    else:
        return 'Best_score'

    
train['creatinine_apache_s'] = train['creatinine_apache'].map(creatinine_apache)
test['creatinine_apache_s'] = test['creatinine_apache'].map(creatinine_apache)        
    
#serum with arf ?????
#urine output 


def urineoutput_apache(x): 
    if pd.isna(x):
        return np.nan
    elif x <= 399:
        return 15
    elif x > 399 and x <= 599: 
        return 8 
    elif x > 599  and x <= 899: 
        return 7
    elif x > 899 and x <= 1499:
        return 5
    elif x > 1499 and x<= 1999:
        return 4
    elif x > 1999 and x<= 3999:
        return 0
    elif x > 3999:
        return 1   
    else:
        return 'Best_score'
    
    
train['urineoutput_apache_s'] = train['urineoutput_apache'].map(urineoutput_apache)
test['urineoutput_apache_s'] = test['urineoutput_apache'].map(urineoutput_apache)        

#serumBUN 

def bun_apache(x): 
    if pd.isna(x):
        return np.nan
    elif x <= 16.9:
        return 0
    elif x > 16.9 and x <= 19: 
        return 2
    elif x > 19  and x <= 39: 
        return 7
    elif x > 39 and x <= 79:
        return 11
    elif x > 79:
        return 12
    else:
        return 'Best_score'

    
train['bun_apache_s'] = train['bun_apache'].map(bun_apache)
test['bun_apache_s'] = test['bun_apache'].map(bun_apache)        

#serumNa 

def sodium_apache(x): 
    if pd.isna(x):
        return np.nan
    elif x <= 119:
        return 3
    elif x > 119 and x <= 134: 
        return 2
    elif x > 134  and x <= 154: 
        return 0
    elif x > 154:
        return 4
    else:
        return 'Best_score'
    
train['sodium_apache_s'] = train['sodium_apache'].map(sodium_apache)
test['sodium_apache_s'] = test['sodium_apache'].map(sodium_apache)        

#serumAlbumin 

def albumin_apache(x): 
    if pd.isna(x):
        return np.nan
    elif x <= 1.9:
        return 11
    elif x > 1.9 and x <= 2.4: 
        return 6 
    elif x > 2.4  and x <= 4.4: 
        return 0
    elif x > 4.4:
        return 4
    else:
        return 'Best_score'


train['albumin_apache_s'] = train['albumin_apache'].map(albumin_apache)
test['albumin_apache_s'] = test['albumin_apache'].map(albumin_apache)        
    
#SerumBilirubin 

def bilirubin_apache(x): 
    if pd.isna(x):
        return np.nan
    elif x <= 1.9:
        return 0
    elif x > 1.9 and x <= 2.9: 
        return 5
    elif x > 2.9  and x <= 4.9: 
        return 6
    elif x > 4.9 and x <= 7.9:
        return 8
    elif x > 7.9:
        return 16
    else:
        return 'Best_score'


train['bilirubin_apache_s'] = train['bilirubin_apache'].map(bilirubin_apache)
test['bilirubin_apache_s'] = test['bilirubin_apache'].map(bilirubin_apache)        
    
#SerumGlucose 

def glucose_apache(x): 
    if pd.isna(x):
        return np.nan
    elif x <= 39:
        return 8
    elif x > 39 and x <= 59: 
        return 9
    elif x > 59  and x <= 199: 
        return 0
    elif x > 199 and x <= 349:
        return 3
    elif x > 349:
        return 5
    else:
        return 'Best_score'


train['glucose_apache_s'] = train['glucose_apache'].map(glucose_apache)
test['glucose_apache_s'] = test['glucose_apache'].map(glucose_apache)        
    
#Age
#Comorbidities


In [71]:
#Acute physiology score
train['APS'] = train['pulse_apache'] + train['meanBP_apache'] + train['temperature_apache'] + \
               train['rr_apache']  + train['pao2_apache_s'] + train['hematocrit_apache_s'] + \
               train['wbc_apache_s'] + train['creatinine_apache_s'] + train['urineoutput_apache_s'] + \
               train['bun_apache_s'] + train['sodium_apache_s'] + train['albumin_apache_s'] + \
               train['bilirubin_apache_s'] + train['glucose_apache_s']

test['APS'] = test['pulse_apache'] + test['meanBP_apache'] + test['temperature_apache'] + \
              test['rr_apache']  + test['pao2_apache_s'] + test['hematocrit_apache_s'] + \
              test['wbc_apache_s'] + test['creatinine_apache_s'] + test['urineoutput_apache_s'] + \
              test['bun_apache_s'] + test['sodium_apache_s'] + test['albumin_apache_s'] + \
              test['bilirubin_apache_s'] + test['glucose_apache_s']    



In [72]:
import math

In [73]:
def freq(df_train, df_test, name):
    tmp = pd.concat([df_train[name], df_test[name]]).value_counts()
    df_train[name+'_freq'] = df_train[name].apply(lambda x: tmp[x] if math.isnan(x) == False else np.nan)
    df_test[name+'_freq']  = df_test[name].apply(lambda x: tmp[x] if math.isnan(x) == False else np.nan)
    return df_train, df_test

train, test = freq(train, test, 'icu_id')
train, test = freq(train, test, 'hospital_id')
#train, test = freq(train, test, 'encounter_id')
#train, test = freq(train, test, 'patient_id')

train, test = freq(train, test, 'ventilated_apache')
train, test = freq(train, test, 'd1_spo2_min')
train, test = freq(train, test, 'gcs_motor_apache')
train, test = freq(train, test, 'd1_arterial_ph_min')

In [74]:
# APACHE-IIIj-scorer
def age_score(x):
    if x < 45:
        return 0
    elif x < 60:
        return 5
    elif x < 65:
        return 11
    elif x < 70:
        return 13
    elif x < 75:
        return 16
    elif x < 85:
        return 17
    else:
        return 24
    
def temp_score(x):
    if x < 32.9:
        return 20
    elif x < 33.4:
        return 16
    elif x < 33.9:
        return 13
    elif x < 34.9:
        return 8
    elif x < 35.9:
        return 2
    elif x < 39.9:
        return 0
    else:
        return 4
    
def hr_score(x):
    if x <= 39:
        return 8
    elif x < 49:
        return 5
    elif x < 99:
        return 0
    elif x < 109:
        return 1
    elif x < 119:
        return 5
    elif x < 139:
        return 7
    elif x < 154:
        return 13
    else:
        return 17

def respiratory_score(x):
    if x <= 5:
        return 17
    elif x < 11:
        return 8
    elif x < 13:
        return 7
    elif x < 24:
        return 0
    elif x < 34:
        return 6
    elif x < 39:
        return 9
    elif x < 49:
        return 11
    else:
        return 18

def map_score(x):
    if x <= 39:
        return 23
    elif x < 59:
        return 15
    elif x < 69:
        return 7
    elif x < 79:
        return 6
    elif x < 99:
        return 0
    elif x < 119:
        return 4
    elif x < 129:
        return 7
    elif x < 139:
        return 9
    else:
        return 10
    
def sodium_score(x):
    if x <=119:
        return 3
    elif x < 135:
        return 2
    elif x < 154:
        return 0
    else:
        return 4

def creatinine_arf_score(x):
    if x < 133:
        return 0
    else:
        return 10

def creatinine_score(x):
    if x < 44:
        return 3
    elif x < 133:
        return 0
    elif x < 171:
        return 4
    else:
        return 7
    
def urinal_score(x):
    if x < 6.2:
        return 0
    elif x < 7.1:
        return 2
    elif x < 14.3:
        return 7
    elif x < 28.5:
        return 11
    else:
        return 12
    
def albumin_score(x):
    if x <= 19:
        return 11
    elif x < 25:
        return 6
    elif x < 44:
        return 0
    else:
        return 4
    
def bilirubin_score(x):
    if x < 35:
        return 0
    elif x < 51:
        return 5
    elif x < 85:
        return 6
    elif x < 135:
        return 8
    else:
        return 16
    
def glucose_score(x):
    if x < 2.2:
        return 8
    elif x < 3.3:
        return 9
    elif x < 11.1:
        return 0
    elif x < 19.3:
        return 3
    else:
        return 5
    
def urine_output_score(x):
    if x <= 399:
        return 15
    elif x < 599:
        return 8
    elif x < 899:
        return 7
    elif x < 1499:
        return 5
    elif x < 1999:
        return 4
    elif x < 3999:
        return 0
    else:
        return 1
    
def wbc_score(x):
    if x < 1:
        return 19
    elif x < 2.9:
        return 5
    elif x < 19.9:
        return 0
    elif x < 24.9:
        return 1
    else:
        return 5
    
def get_apache_3j(df):
    score = df['age'].map(age_score) + \
            df['temp_apache'].map(temp_score) + \
            df['heart_rate_apache'].map(hr_score) + \
            df['resprate_apache'].map(respiratory_score) + \
            df['map_apache'].map(map_score) + \
            df['sodium_apache'].map(sodium_score) + \
            df['arf_apache'] * df['creatinine_apache'].map(creatinine_arf_score) + \
            (1 - df['arf_apache']) * df['creatinine_apache'].map(creatinine_score) + \
            df['bun_apache'].map(urinal_score) + \
            df['albumin_apache'].map(albumin_score) + \
            df['bilirubin_apache'].map(bilirubin_score) + \
            df['glucose_apache'].map(glucose_score) + \
            df['urineoutput_apache'].map(urine_output_score) + \
            df['wbc_apache'].map(wbc_score) + \
            df['aids'] * 23 + df['hepatic_failure'] * 16 + df['lymphoma'] * 13 + df['solid_tumor_with_metastasis'] * 11 + \
            df['leukemia'] * 10 + df['immunosuppression'] * 10 + df['cirrhosis'] * 4
    
    return score

In [75]:
def platelets_sofa_score(x):
    if x < 20:
        return 4
    elif x < 50:
        return 3
    elif x < 100:
        return 2
    elif x < 150:
        return 1
    else:
        return 0
    
def bilirubin_sofa_score(x):
    if x < 20:
        return 0
    elif x < 32:
        return 1
    elif x < 101:
        return 2
    elif x < 204:
        return 3
    else:
        return 4
    
def cardio_sofa_score(x):
    if x >= 70:
        return 0
    else:
        return 1
    
def gcs_sofa_score(x):
    if x > 14:
        return 0
    elif x > 12:
        return 1
    elif x > 9:
        return 2
    elif x > 5:
        return 1
    else:
        return 0

def urine_sofa_score(x):
    if x <= 200:
        return 4
    elif x < 500:
        return 3
    elif x < 700:
        return 2
    elif x < 1200:
        return 1
    else:
        return 0
    
def get_sofa(df):
    score = df['h1_platelets_max'].map(platelets_sofa_score) + \
            df['bilirubin_apache'].map(bilirubin_sofa_score) + \
            df['map_apache'].map(cardio_sofa_score) + \
            df['Glasglow_comma_score'].map(gcs_sofa_score) + \
            df['urineoutput_apache'].map(urine_sofa_score)
    
    return score

In [76]:
train['heart_rate_max_rate_diff'] = (200 - train['age']) - train['heart_rate_apache']
test['heart_rate_max_rate_diff'] = (200 - test['age']) - test['heart_rate_apache']

train['apache_3j_score'] = get_apache_3j(train)
test['apache_3j_score'] = get_apache_3j(test)

train['sofa_3_score'] = get_sofa(train)
test['sofa_3_score'] = get_sofa(test)

train['GFR'] = 175 * (train['creatinine_apache'] ** (-1.154)) * (train['age'] ** (-0.203)) * train['gender'].apply(lambda x: 1 if x == 'M' else 0.742)
test['GFR'] = 175 * (test['creatinine_apache'] ** (-1.154)) * (test['age'] ** (-0.203)) * train['gender'].apply(lambda x: 1 if x == 'M' else 0.742)

In [77]:
train['new1'] = train['bun_apache'] + train['age'] + train['d1_hco3_min'] +  train['d1_lactate_min'] + train['d1_arterial_ph_min'] + train['wbc_apache']
test['new1'] = test['bun_apache'] + test['age'] + test['d1_hco3_min'] +  test['d1_lactate_min'] + test['d1_arterial_ph_min'] + test['wbc_apache']

In [78]:
train['new2'] = train['bun_apache'] / train['creatinine_apache']
test['new2'] = test['bun_apache'] / test['creatinine_apache']

In [79]:
train['hr_min_max_diff'] = train['h1_heartrate_max'] - train['h1_heartrate_min']
test['hr_min_max_diff'] = test['h1_heartrate_max'] - test['h1_heartrate_min']

train['lactate_min_max_diff'] = train['h1_lactate_max'] - train['h1_lactate_min']
test['lactate_min_max_diff'] = test['h1_lactate_max'] - test['h1_lactate_min']

In [80]:
#DROP_COLS = ['readmission_status', 'encounter_id', 'patient_id', 'hospital_id', 'icu_id']

#train.drop(DROP_COLS, axis=1, inplace=True)
#test.drop(DROP_COLS, axis=1, inplace=True)

In [82]:
train.to_csv("train6.csv", index=False)
test.to_csv("test6.csv", index=False)